In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
import socket, struct


# def ip2int(ip):
#     # print(ip)
#     packedIP = socket.inet_aton(ip)
#     return struct.unpack("!L", packedIP)[0]

def ip2int(ip):
    try:
        packedIP = socket.inet_aton(ip)
        return struct.unpack("!L", packedIP)[0]
    except (socket.error, OSError):
        return np.nan

def anonymize_ip(ip_series):
        anonymized_ip = {}
        new_ip = 1
        result = []
        for ip in ip_series:
            if ip not in anonymized_ip:
                anonymized_ip[ip] = new_ip
                new_ip += 1
            result.append(anonymized_ip[ip])
        return result


def anonymize_port(port_series):
        anonymized_port = {}
        new_port = 1
        result = []
        for port in port_series:
            if port not in anonymized_port:
                anonymized_port[port] = new_port
                new_port += 1
            result.append(anonymized_port[port])
        return result

root_folder = "/home/atg205/Malware/CICIot/"


In [39]:

X_data = []
y_data = []



for root, dirs, files in os.walk(root_folder):
    for file in files:
        if not file.endswith(".csv"):
             continue
        file_path = os.path.join(root,file)
        df = pd.read_csv(file_path)

        df.columns = df.columns.str.strip()
        # df.drop(columns=[" Label"], inplace = True)
        # df.drop(columns=["Flow ID", "Timestamp", "Label"], inplace = True)
        df.drop(columns=["label1", "label2", "label3", "label_full",
                         "network_ips_all",
                         "network_macs_dst",
                         "network_ports_all",
                         "device_name",
                         "network_ips_dst",
                         "network_ips_src",
                         "network_macs_all",
                         "network_macs_src",
                         "network_ports_dst",
                         "network_ports_src",
                         "network_protocols_src",
                         "network_protocols_all",
                         "network_protocols_dst",
                         "log_data-types"
                         ], inplace = True)
  

        df['device_mac'] = anonymize_ip(df['device_mac'])       

        X = df.drop(columns="label4")
        y = df['label4']

        X_data.append(X)
        y_data.append(y)



In [40]:

X_df = pd.concat(X_data,axis=0)
# Parse timestamps (UTC-aware)
X_df["timestamp_start"] = pd.to_datetime(X_df["timestamp_start"], utc=True)
X_df["timestamp_end"] = pd.to_datetime(X_df["timestamp_end"], utc=True)

# Time since first observation PER MAC address (seconds)
X_df["t_since_first_s"] = (
    X_df.groupby("device_mac")["timestamp_start"]
      .transform(lambda x: (x - x.min()).dt.total_seconds())
)

# Observation duration (milliseconds)
X_df["obs_duration_ms"] = (
    X_df["timestamp_end"] - X_df["timestamp_start"]
).dt.total_seconds() * 1000

# Drop raw timestamp columns (keep only numeric features)
X_df = X_df.drop(columns=[
    "timestamp",
    "timestamp_start",
    "timestamp_end"
])

y_df = pd.concat(y_data, axis=0)
y_df

0         ddos_syn-flood-port-80
1         ddos_syn-flood-port-80
2         ddos_syn-flood-port-80
3         ddos_syn-flood-port-80
4         ddos_syn-flood-port-80
                   ...          
136795                    benign
136796                    benign
136797                    benign
136798                    benign
136799                    benign
Name: label4, Length: 227191, dtype: object

In [41]:
X_df

,device_mac,log_data-ranges_avg,log_data-ranges_max,log_data-ranges_min,log_data-ranges_std_deviation,log_data-types_count,log_interval-messages,log_messages_count,network_fragmentation-score,network_fragmented-packets,...,network_ttl_avg,network_ttl_max,network_ttl_min,network_ttl_std_deviation,network_window-size_avg,network_window-size_max,network_window-size_min,network_window-size_std_deviation,t_since_first_s,obs_duration_ms
0,1,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999737e+05,1000.0
1,1,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999747e+05,1000.0
2,1,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999757e+05,1000.0
3,1,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999767e+05,1000.0
4,1,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.999777e+05,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136795,38,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.048428e+07,1000.0
136796,38,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,255.0,255.0,255.0,0.0,2575.0,2575.0,2575.0,0.0,2.048428e+07,1000.0
136797,38,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,246.5,255.0,238.0,8.5,34020.5,65535.0,2506.0,31514.5,2.048428e+07,1000.0
136798,38,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.048428e+07,1000.0


In [43]:
output = '/home/atg205/Malware/CICIot/processed_data'


X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42, stratify=y_df)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


np.save(os.path.join(output, 'X_train.npy'), X_train)
np.save(os.path.join(output, 'y_train.npy'), y_train)
np.save(os.path.join(output, 'X_test.npy'), X_test)
np.save(os.path.join(output, 'y_test.npy'), y_test)
np.save(os.path.join(output, 'X_valid.npy'), X_valid)
np.save(os.path.join(output, 'y_valid.npy'), y_valid)

In [46]:
X_train

array([[ 1.5288829 , -0.26747234, -0.26908024, ..., -0.61668403,
        -1.35032997,  0.        ],
       [ 0.40829228, -0.26747234, -0.26908024, ..., -0.63747266,
        -1.2962548 ,  0.        ],
       [ 0.22152718, -0.26747234, -0.26908024, ..., -0.66929957,
         0.79087787,  0.        ],
       ...,
       [ 1.80903055, -0.26747234, -0.26908024, ..., -0.66929957,
         0.79088606,  0.        ],
       [-1.08582854, -0.15834282, -0.16075414, ...,  1.69511825,
         0.79118061,  0.        ],
       [-0.05862048, -0.26747234, -0.26908024, ..., -0.48328349,
        -1.3678364 ,  0.        ]], shape=(181752, 74))